In [36]:
import requests
import pandas as pd
import json


# Opeing the json file I made ealier for another chart, as it has relevant infomrtion
with open('Tax vs Spending per capita data.json') as f:
    data = json.load(f)
df = pd.DataFrame(data)


# Scraping the table from the website, and making it a dataframe

url = "https://rankedex.com/society-rankings/education-index"

response = requests.get(url)
df_Education = pd.read_html(response.text)[0]

df_Education


,Unnamed: 0,Rank,Education index
0,Germany,1,0.940
1,Australia,2,0.929
2,Denmark,3,0.920
3,Ireland,4,0.918
4,New Zealand,5,0.917
...,...,...,...
184,South Sudan,157,0.297
185,Mali,158,0.293
186,Burkina Faso,159,0.286
187,Eritrea,160,0.281


In [37]:
import pycountry

# Using the pycountry code to assing country code values to the country names in order to make merging with the other data easier.
# Using country codes prevents problems from different datasets using slightly different names for the same countrys.

def do_fuzzy_search(country):
    try:
        result = pycountry.countries.search_fuzzy(country)
        return result[0].alpha_3
    except:
        return "Attention has not worked"
# Making it obvioous where it has not worked

iso_map = {country: do_fuzzy_search(country)
           for country in df_Education["Unnamed: 0"].unique()}
df_Education["Country code"] = df_Education["Unnamed: 0"].map(iso_map)

# Manually assigning the values for where pycountry failed.
df_Education.loc[102, 'Country code'] = 'PSE'
df_Education.loc[132, 'Country code'] = 'CPV'
df_Education.loc[138, 'Country code'] = 'SWZ'
df_Education.loc[144, 'Country code'] = 'TMP'
df_Education.loc[150, 'Country code'] = 'COD'
df_Education.loc[152, 'Country code'] = 'LAO'
df_Education.loc[166, 'Country code'] = 'CIV'


df_Education.head()


,Unnamed: 0,Rank,Education index,Country code
0,Germany,1,0.940,DEU
1,Australia,2,0.929,AUS
2,Denmark,3,0.920,DNK
3,Ireland,4,0.918,IRL
4,New Zealand,5,0.917,NZL


In [38]:
# Merging the two datasets together by their country codes.

df_merged = pd.merge(df_Education, df, on='Country code')



In [39]:
# Dropping unwanted column

df_merged = df_merged.drop(columns=['Unnamed: 0', 'Rank'])
df_merged.head()


,Education index,Country code,Country,Population,GDP,GDP per capita,Total tax revenue,Government spending,Spending per Capita,Tax per Capita
0,0.940,DEU,Germany,83237124.0,4.262767e+12,51238.0,1.666454e+12,2.362676e+12,28384.882688,20020.561979
1,0.929,AUS,Australia,25767000.0,1.635255e+12,63464.0,4.595460e+11,5.938560e+11,23047.153336,17834.672255
2,0.920,DNK,Denmark,5873420.0,3.983030e+11,68202.0,1.802870e+11,1.938310e+11,33001.385905,30695.404041
3,0.918,IRL,Ireland,5060005.0,5.045170e+11,100129.0,9.322100e+10,1.171490e+11,23151.953407,18423.104325
4,0.917,NZL,New Zealand,5112000.0,2.469740e+11,48317.0,7.253200e+10,9.043100e+10,17689.945227,14188.575900


In [40]:
# Scraping website for life expectancy data

Life = pd.read_html(
    'https://www.worldometers.info/demographics/life-expectancy/')
Life[0]

df_Life = Life[0]
df_Life = df_Life.rename(
    columns={'Life Expectancy (both sexes)': 'Life expectancy'})

df_Life = df_Life[['Life expectancy', 'Country']]


# Doing the same, assinging country codes for the Life expectancy dataset.
def do_fuzzy_search(country):
    try:
        result = pycountry.countries.search_fuzzy(country)
        return result[0].alpha_3
    except:
        return "Attention has not worked"


iso_map = {country: do_fuzzy_search(country)
           for country in df_Life["Country"].unique()}
df_Life["Country code"] = df_Life["Country"].map(iso_map)


# Adding the remaining codes manually
df_Life.loc[8, 'Country code'] = 'GGY'
df_Life.loc[34, 'Country code'] = 'VIR'
df_Life.loc[43, 'Country code'] = 'CZE'
df_Life.loc[120, 'Country code'] = 'VCT'
df_Life.loc[139, 'Country code'] = 'STP'
df_Life.loc[149, 'Country code'] = 'LAO'
df_Life.loc[188, 'Country code'] = 'COD'

df_Life = df_Life.drop('Country', axis=1)

# Merging the dataframes
df_merged = pd.merge(df_Life, df_merged, on='Country code')

# Removing duplicates
df_merged = df_merged.drop_duplicates(subset='Country code')
df_merged = df_merged.reset_index(drop=True)

df_merged.head(10)

,Life expectancy,Country code,Education index,Country,Population,GDP,GDP per capita,Total tax revenue,Government spending,Spending per Capita,Tax per Capita
0,85.03,JPN,0.848,Japan,125681593.0,4.932556e+12,39246.0,1.334944e+12,1.609710e+12,12807.842116,10621.634944
1,84.25,CHE,0.897,Switzerland,8736510.0,7.997960e+11,91546.0,2.344450e+11,2.642320e+11,30244.571345,26835.086322
2,84.07,SGP,0.832,Singapore,5454000.0,3.969920e+11,72795.0,5.967800e+10,9.603100e+10,17607.444078,10942.060873
3,84.01,ITA,0.791,Italy,58983122.0,2.101275e+12,35625.0,6.207390e+11,9.407710e+11,15949.833920,10524.010581
4,83.99,ESP,0.824,Spain,47615034.0,1.426224e+12,30090.0,4.819450e+11,6.577500e+11,13813.914320,10121.698117
5,83.94,AUS,0.929,Australia,25767000.0,1.635255e+12,63464.0,4.595460e+11,5.938560e+11,23047.153336,17834.672255
6,83.52,ISL,0.912,Iceland,376248.0,2.545900e+10,67665.0,9.962000e+09,9.735000e+09,25873.891688,26477.217155
7,83.50,KOR,0.862,South Korea,51736000.0,1.797810e+12,34744.0,3.631200e+11,4.144520e+11,8010.901500,7018.710376
8,83.49,ISR,0.874,Israel,9367000.0,4.885270e+11,52152.0,1.288650e+11,1.784650e+11,19052.524821,13757.339596
9,83.33,SWE,0.904,Sweden,10452326.0,6.356640e+11,60816.0,2.510090e+11,2.822230e+11,27000.975668,24014.654729


In [41]:
import json
# Exporting the merged dataframe as a json file

exported_values = list(df_merged.T.to_dict().values())
open('Gov spending, Education and Life expectancy.json',
     'w').write(json.dumps(exported_values))


58256